**JsonOutputParser**

    - JSON 스키마를 지정할 수 있어 해당 스키마에 맞게 LLM 에서 데이터를 조회하여 결과를 도출함
    - 데이터 모델을 정의할 때는 Pydantic과 같은 도구를 사용해서 스키마가 잘 정의되고 검증될 수 있도록 한다.
    - 사용자가 특정 데이터 구조를 기반으로 정보를 요청하고 모델이 그 요구사항에 맞는 데이터를 정확히 제공할 수 있게 된다.

In [1]:
from dotenv import load_dotenv

In [2]:
from typing import List

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field


In [3]:
model = ChatOpenAI(
    temperature=0,
    model='gpt-3.5-turbo'
)

In [6]:
# 원하는 데이터 구조

class Topic(BaseModel):
    description: str = Field(description="Concise description about topic")
    hashtags: str = Field(description="Some keywords in hashtag format")

    - JsonOutputParser를 사용해서 파서를 설정, 프롬프트 템플릿에 지시사항 주입
    - PrompTemplate을 이용해 사용자 질의에 대한 답변을 생성하는 템플릿 구성
        (질의 : query, 포맷 지시사항 : from_instructions)
    - prompt, model, parser를 chain으로 구성
    - 처리 체인 호출해 query에 대한 응답 생성

In [8]:
query = 'pydantic에 대해 알려주세요'

parser = JsonOutputParser(pydantic_object=Topic)

prompt = PromptTemplate(
    template = 'Answer the user query.\n{format_instruction}\n{query}\n',
    input_variables=['query'],
    partial_variables={'format_instruction':parser.get_format_instructions()},
)

prompt

PromptTemplate(input_variables=['query'], partial_variables={'format_instruction': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"description": {"title": "Description", "description": "Concise description about topic", "type": "string"}, "hashtags": {"title": "Hashtags", "description": "Some keywords in hashtag format", "type": "string"}}, "required": ["description", "hashtags"]}\n```'}, template='Answer the user query.\n{format_instruction}\n{query}\n')

    PromptTemplate(input_variables=['query'], partial_variables={'format_instruction': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"description": {"title": "Description", "description": "Concise description about topic", "type": "string"}, "hashtags": {"title": "Hashtags", "description": "Some keywords in hashtag format", "type": "string"}}, "required": ["description", "hashtags"]}\n```'}, template='Answer the user query.\n{format_instruction}\n{query}\n')

In [9]:
chain = prompt | model | parser
chain

PromptTemplate(input_variables=['query'], partial_variables={'format_instruction': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"description": {"title": "Description", "description": "Concise description about topic", "type": "string"}, "hashtags": {"title": "Hashtags", "description": "Some keywords in hashtag format", "type": "string"}}, "required": ["description", "hashtags"]}\n```'}, template='Answer the user query.\n{format_instruction}\n{query}\n')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x11ff2b0d0>, async_client=<openai.

In [10]:
response = chain.invoke({'query':query})
response

{'description': 'pydantic is a data validation and settings management using Python type annotations.',
 'hashtags': '#pydantic #python #datavalidation #settingsmanagement'}

**Without Pydantic**

    - Pydantic 없이 해당 기능 구현
    - 이 경우 JSON을 반환하지만 스키마가 어떻게 되어야 하는지에 대한 구체적인 정보 제공하지 않음

In [18]:
# JsonOutputParser 클래스의 인스턴스를 parser 변수에 할당
# PromptTemplate 클래스로 사용자 쿼리에 대한 프롬프트 생성
# (template : 사용자 쿼리 처리할 템플릿 문자열 , input_variables= 사용할 변수 목록, partial_variables = parser에서 제공할 형식 지침)
# prompt, model, parser로 chain 생성
# chain.invoke로 쿼리에 대한 수행

query = "python의 pydantic에 대해 설명해주세요. pydantic에 대한 설명은 `description`에 관련 키워드는 `hashtags에 담아주세요. 한국말로 응답해주세요"

parser = JsonOutputParser()

prompt = PromptTemplate(
    template = "Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=['query'],
    partial_variables={'format_instructions' : parser.get_format_instructions()},
    
)

chain = prompt | model | parser
chain

PromptTemplate(input_variables=['query'], partial_variables={'format_instructions': 'Return a JSON object.'}, template='Answer the user query.\n{format_instructions}\n{query}\n')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x11ff2b0d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x11ff547f0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')
| JsonOutputParser()

In [19]:
chain.invoke({'query':query})

{'description': 'Python 데이터 유효성 검사 라이브러리, 데이터 모델 정의 및 유효성 검사에 사용됨',
 'hashtags': ['Python', 'pydantic', '데이터 유효성 검사']}